In [ ]:
%load_ext blackcellmagic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

from datetime import datetime
import json

In [2]:
import os

os.chdir("data/")

In [3]:
items = pd.read_csv("items.csv", delimiter="\|", engine="python")
items_c = pd.read_csv("items_stopc.csv", delimiter="\|", engine="python")
items_dedup_c = pd.read_csv("items_dedup_stopc.csv", delimiter="\|", engine="python")
eval_ = pd.read_csv("evaluation.csv", delimiter="\|", engine="python")

convert_dict = {"itemID": np.int64}

eval_ = eval_.astype(convert_dict)

In [4]:
amazon_val = pd.read_csv("../dmc21_amazon_validation.csv", engine="python")
amazon_test = pd.read_csv("../dmc21_amazon_test.csv", engine="python")

In [5]:
amazon_val_ids = amazon_val["itemID"].values.tolist()
amazon_val_ids = [np.int64(a) for a in amazon_val_ids]
len(amazon_val_ids)

232

In [6]:
amazon_test_ids = amazon_test["itemID"].values.tolist()
amazon_test_ids = [np.int64(i) for i in amazon_test_ids]
len(amazon_test_ids)

233

In [7]:
def create_embeddings(path: str) -> dict:
    #if emb not in ["title", "book", "author", "publisher", "topic"]:
    #    raise Exception("Not known embeddings")

    #path = "embeddings/%s_embeddings_multilingual.npz" % emb

    embeddings = np.load(path)["arr_0"]

    item_ids = items_c["itemID"].values.tolist()
    item_ids = [np.int64(i) for i in item_ids]

    emb_dict = {}

    for i in range(len(item_ids)):
        emb_dict[item_ids[i]] = embeddings[i]

    return emb_dict

In [8]:
book_emb = create_embeddings(path="embeddings/book_embeddings.npz")
title_emb = create_embeddings(path="embeddings/title_embeddings.npz")
author_emb = create_embeddings(path="embeddings/author_embeddings.npz")
publisher_emb = create_embeddings(path="embeddings/publisher_embeddings.npz")
topic_emb = create_embeddings(path="embeddings/topic_embeddings.npz")

In [9]:
special_chars = [
    "\n",
    "\r",
    "!",
    '"',
    "#",
    "$",
    "%",
    "&",
    "'",
    "(",
    ")",
    "*",
    "+",
    ",",
    "-",
    ".",
    "/",
    ":",
    ";",
    "=",
    ">",
    "?",
    "@",
    "[",
    "]",
    "_",
    "`",
    "|",
    "~",
    "\x81",
    "\x8e",
    "\x92",
    "\xa0",
    "¡",
    "©",
    "«",
    "¬",
    "®",
    "°",
    "¶",
    "»",
    "÷",
]

replace_chars = {
    "¹": "1",
    "²": "2",
    "³": "3",
    "à": "a",
    "á": "a",
    "â": "a",
    "ã": "a",
    "å": "a",
    "æ": "a",
    "ç": "c",
    "è": "e",
    "é": "e",
    "ê": "e",
    "ë": "e",
    "ì": "i",
    "í": "i",
    "î": "i",
    "ï": "i",
    "ñ": "n",
    "ò": "o",
    "ó": "o",
    "ô": "o",
    "õ": "o",
    "ø": "o",
    "ù": "u",
    "ú": "u",
    "û": "u",
    "ý": "y",
    "ÿ": "y",
}

In [10]:
def cleanse_string(string_: str):
    substrings = string_.split()
    substrings[:] = [s.lower() for s in substrings]
    for sc in special_chars:
        substrings[:] = [s.replace(sc, "") for s in substrings]
    for key, val in replace_chars.items():
        substrings[:] = [s.replace(key, val) for s in substrings]
    substrings = list(filter(None, substrings))
    return " ".join(substrings)

In [6]:
with open("publisher_dict.json") as f:
    data = f.read()
publisher_dict = json.loads(data)

In [12]:
publisher_dict_2 = dict()
for k, v in publisher_dict.items():
    topic = cleanse_string(str(k))
    publisher_dict_2[topic] = cleanse_string(str(v))

In [13]:
publisher_dict_2["yfh"]

'children’s teenage fiction fantasy'

In [14]:
items_c[items_c["itemID"] == np.int64(445)]

,itemID,title,author,publisher,main topic,subtopics
440,445,phantastische reise,isaac asimov,epubli,fl,NaN


In [15]:
def cosine_sim(embeddings: dict, book_id_1: np.int64(), book_id_2: np.int64) -> float:
    sim = float(0)
    embeddings = embeddings
    sim = cosine(embeddings[book_id_1], embeddings[book_id_2])
    return round(1 - sim, 2)

In [16]:
cosine_sim(embeddings=book_emb, book_id_1=np.int64(75539), book_id_2=np.int64(666))

0.52

In [17]:
from pyjarowinkler import distance


def jaro_winkler_sim(string_1: str, string_2: str) -> float:
    return distance.get_jaro_distance(string_1, string_2, winkler=True)

In [19]:
test_eval = eval_.at[420, "itemID"]
test_eval

75539

In [38]:
def get_recommendations(item_id: np.int64(), **kwargs) -> tuple:
    attributes_ = {}
    similarities = []

    book_entry = items.loc[items["itemID"] == item_id].reset_index()
    book_entry_c = items_c[items_c["itemID"] == item_id].reset_index()

    author = str(book_entry_c.at[0, "author"])
    title = str(book_entry_c.at[0, "title"])
    publisher = str(book_entry_c.at[0, "publisher"])
    topic = str(book_entry_c.at[0, "main topic"])
    #if str(book_entry_c.at[0, "main topic"]) != "nan":
    #    topic = publisher_dict_2[str(book_entry_c.at[0, "main topic"])]
    #else:
    #    topic = str(book_entry_c.at[0, "main topic"])

    for k, v in kwargs.items():
        attributes_[k] = v

    weights_sum = 0
    for k, v in attributes_.items():
        weights_sum += attributes_[k][0]
    if round(weights_sum, 1) != 1.0:
        raise Exception("sum of weights unequal to 1.0")

    for i in items_dedup_c.index:
        book_id_comp = items_dedup_c.at[i, "itemID"]

        author_comp = str(items_dedup_c.at[i, "author"])
        title_comp = str(items_dedup_c.at[i, "title"])
        publisher_comp = str(items_dedup_c.at[i, "publisher"])
        topic_comp = str(items_dedup_c.at[i, "main topic"])
        #if str(items_dedup_c.at[i, "main topic"]) != "nan":
        #    topic_comp = publisher_dict_2[str(items_dedup_c.at[i, "main topic"])]
        #else:
        #    topic_comp = str(items_dedup_c.at[i, "main topic"])

        author_sim = 0
        title_sim = 0
        publisher_sim = 0
        topic_sim = 0
        book_sim = 0

        sim_scores = []

        if "book" in attributes_.keys():
            book_sim = cosine_sim(
                embeddings=book_emb, book_id_1=item_id, book_id_2=np.int64(book_id_comp)
            )
        else:
            if "author" in attributes_.keys():
                if attributes_["author"][1] == cosine_sim:
                    author_sim = cosine_sim(embeddings=author_emb, book_id_1=item_id, book_id_2=np.int64(book_id_comp))
                    sim_scores.append(tuple([attributes_["author"][0], author_sim]))
                else:
                    author_sim = attributes_["author"][1](author, author_comp)
                    sim_scores.append(tuple([attributes_["author"][0], author_sim]))
            if "title" in attributes_.keys():
                if attributes_["title"][1] == cosine_sim:
                    title_sim = cosine_sim(embeddings=title_emb, book_id_1=item_id, book_id_2=np.int64(book_id_comp))
                    sim_scores.append(tuple([attributes_["title"][0], title_sim]))
                else:
                    title_sim = attributes_["title"][1](title, title_comp)
                    sim_scores.append(tuple([attributes_["title"][0], title_sim]))
            if "publisher" in attributes_.keys():
                if attributes_["publisher"][1] == cosine_sim:
                    publisher_sim = cosine_sim(embeddings=publisher_emb, book_id_1=item_id, book_id_2=np.int64(book_id_comp))
                    sim_scores.append([attributes_["publisher"][0], publisher_sim])
                else:
                    publisher_sim = attributes_["publisher"][1](
                        publisher, publisher_comp
                    )
                    sim_scores.append(
                        tuple([attributes_["publisher"][0], publisher_sim])
                    )
            if "topic" in attributes_.keys():
                if attributes_["topic"][1] == cosine_sim:
                    topic_sim = cosine_sim(embeddings=topic_emb, book_id_1=item_id, book_id_2=np.int64(book_id_comp))
                    sim_scores.append(tuple([attributes_["topic"][0], topic_sim]))
                else:
                    topic_sim = attributes_["topic"][1](topic, topic_comp)
                    sim_scores.append(tuple([attributes_["topic"][0], topic_sim]))

        if "book" not in attributes_.keys():
            for sim_score in sim_scores:
                book_sim += sim_score[0] * sim_score[1]

        if book_sim == float(1):
            continue
        if len(similarities) < 7:
            similarities.append(tuple([book_id_comp, book_sim]))
        else:
            book_sims = [x[1] for x in similarities]
            if all(book_sim >= s[1] for s in similarities):
                similarities.append(tuple([book_id_comp, book_sim]))
            similarities.sort(key=lambda s: s[1], reverse=True)
            similarities = similarities[:7]

    recommendations = items[items["itemID"].isin([s[0] for s in similarities])]
    recommendations["sim"] = np.nan
    for e in [s for s in similarities]:
        recommendations.loc[(recommendations["itemID"] == e[0]), "sim"] = e[1]
    return book_entry, recommendations.sort_values(by=["sim"], ascending=False)

In [ ]:
# r = get_recommendations(test_eval, author=(0.25, jaro_winkler_sim), title=(0.25, jaro_winkler_sim), topic=(0.25, jaro_winkler_sim), publisher=(0.25, jaro_winkler_sim))
# r = get_recommendations(test_eval, title=((1/3), cosine_sim), author=((1/3), cosine_sim), topic=((1/3), cosine_sim))
r = get_recommendations(test_eval, book=(1.0, cosine_sim))
display(r[0])
display(r[1])

In [93]:
eval_books=eval_["itemID"].values.tolist()
eval_books=[np.int64(a) for a in eval_books]
len(eval_books)

1000

In [105]:
book_val_dict = dict()

In [106]:
for i, e in enumerate(eval_books):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, author=(0.25, jaro_winkler_sim), title=(0.25, jaro_winkler_sim), topic=(0.25, jaro_winkler_sim), publisher=(0.25, jaro_winkler_sim))
    r = recommendations[1]["itemID"].values.tolist()
    book_val_dict[e]=r

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [109]:
len(book_val_dict)

1000

In [111]:
from evaluation_workflow import evaluate_recommender
evaluate_recommender(book_val_dict, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 36.21%
Recall@1: 12.93%
F1-Measure@1: 17.07%
-------
Precision@2: 26.72%
Recall@2: 16.77%
F1-Measure@2: 18.34%
-------
Precision@3: 19.83%
Recall@3: 18.1%
F1-Measure@3: 16.9%
-------
Precision@5: 12.67%
Recall@5: 19.2%
F1-Measure@5: 13.74%
-------
Precision@6: 10.63%
Recall@6: 19.31%
F1-Measure@6: 12.42%
-------
Precision@7: 9.11%
Recall@7: 19.31%
F1-Measure@7: 11.28%
-------
Mean Average Precision: 17.16%


In [115]:
evaluate_recommender(data, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 36.21%
Recall@1: 12.93%
F1-Measure@1: 17.07%
-------
Precision@2: 26.72%
Recall@2: 16.77%
F1-Measure@2: 18.34%
-------
Precision@3: 19.83%
Recall@3: 18.1%
F1-Measure@3: 16.9%
-------
Precision@5: 12.67%
Recall@5: 19.2%
F1-Measure@5: 13.74%
-------
Precision@6: 10.63%
Recall@6: 19.31%
F1-Measure@6: 12.42%
-------
Precision@7: 9.11%
Recall@7: 19.31%
F1-Measure@7: 11.28%
-------
Mean Average Precision: 17.16%


In [23]:
book_val_dict2 = dict()

In [24]:
for i, e in enumerate(amazon_val_ids):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, book=(1.0, cosine))
    r = recommendations[1]["itemID"].values.tolist()
    book_val_dict2[e]=r

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [26]:
from evaluation_workflow import evaluate_recommender
evaluate_recommender(book_val_dict2, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 22.84%
Recall@1: 7.11%
F1-Measure@1: 9.97%
-------
Precision@2: 17.24%
Recall@2: 10.47%
F1-Measure@2: 11.77%
-------
Precision@3: 13.22%
Recall@3: 11.59%
F1-Measure@3: 11.2%
-------
Precision@5: 9.22%
Recall@5: 12.9%
F1-Measure@5: 9.86%
-------
Precision@6: 7.97%
Recall@6: 13.6%
F1-Measure@6: 9.22%
-------
Precision@7: 7.08%
Recall@7: 14.03%
F1-Measure@7: 8.67%
-------
Mean Average Precision: 10.98%


In [30]:
book_emb_recom = dict()

In [32]:
for i, e in enumerate(eval_books):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, author=(0.25, jaro_winkler_sim), title=(0.25, jaro_winkler_sim), topic=(0.25, jaro_winkler_sim), publisher=(0.25, jaro_winkler_sim))
    r = recommendations[1]["itemID"].values.tolist()
    book_emb_recom[e]=r

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [33]:
len(book_emb_recom)

1000

In [ ]:
## jaro winkler standard

In [34]:
from evaluation_workflow import evaluate_recommender
evaluate_recommender(book_emb_recom, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 4.74%
Recall@1: 1.42%
F1-Measure@1: 2.03%
-------
Precision@2: 12.5%
Recall@2: 8.75%
F1-Measure@2: 9.15%
-------
Precision@3: 10.92%
Recall@3: 10.62%
F1-Measure@3: 9.64%
-------
Precision@5: 7.16%
Recall@5: 11.16%
F1-Measure@5: 7.88%
-------
Precision@6: 6.03%
Recall@6: 11.27%
F1-Measure@6: 7.15%
-------
Precision@7: 5.17%
Recall@7: 11.27%
F1-Measure@7: 6.48%
-------
Mean Average Precision: 6.42%


In [ ]:
## book embeddings standard

In [32]:
from evaluation_workflow import evaluate_recommender
evaluate_recommender(book_emb_recom, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 22.84%
Recall@1: 7.11%
F1-Measure@1: 9.97%
-------
Precision@2: 17.24%
Recall@2: 10.47%
F1-Measure@2: 11.77%
-------
Precision@3: 13.22%
Recall@3: 11.59%
F1-Measure@3: 11.2%
-------
Precision@5: 9.22%
Recall@5: 12.9%
F1-Measure@5: 9.86%
-------
Precision@6: 7.97%
Recall@6: 13.6%
F1-Measure@6: 9.22%
-------
Precision@7: 7.08%
Recall@7: 14.03%
F1-Measure@7: 8.67%
-------
Mean Average Precision: 10.98%


In [15]:
book_emb_recom_new = dict()

In [16]:
for i, e in enumerate(eval_books):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, book=(1.0, cosine_sim))
    r = sorted(recommendations[1]["itemID"].values.tolist())
    book_emb_recom_new[e]=r

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [28]:
book_emb_jaro = dict()

In [29]:
for i, e in enumerate(eval_books):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, author=(0.25, jaro_winkler_sim), title=(0.25, jaro_winkler_sim), topic=(0.25, jaro_winkler_sim), publisher=(0.25, jaro_winkler_sim))
    r = sorted(recommendations[1]["itemID"].values.tolist())
    book_emb_jaro[e]=r

0
10
20


KeyboardInterrupt: 

In [ ]:
for i, e in enumerate(eval_books):
    if i % 10 == 0:
        print(i)
    recommendations = get_recommendations(e, book=(1.0, cosine_sim))
    r = sorted(recommendations[1]["itemID"].values.tolist())
    book_emb_recom_new[e]=r

In [21]:
from evaluation_workflow import evaluate_recommender

In [40]:
evaluate_recommender(book_emb_recom, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 2.59%
Recall@1: 0.56%
F1-Measure@1: 0.88%
-------
Precision@2: 2.37%
Recall@2: 0.97%
F1-Measure@2: 1.32%
-------
Precision@3: 2.73%
Recall@3: 1.65%
F1-Measure@3: 1.98%
-------
Precision@5: 3.88%
Recall@5: 5.09%
F1-Measure@5: 4.0%
-------
Precision@6: 4.67%
Recall@6: 7.49%
F1-Measure@6: 5.27%
-------
Precision@7: 7.08%
Recall@7: 14.03%
F1-Measure@7: 8.67%
-------
Mean Average Precision: 3.98%


In [22]:
evaluate_recommender(book_emb_recom_new, "dmc21_amazon_validation.csv")

Number of books with recommendations in validation set: 232 of 232
Average number of recommendations per book in validation set: 3.9
Precision@1: 0.86%
Recall@1: 0.3%
F1-Measure@1: 0.43%
-------
Precision@2: 0.43%
Recall@2: 0.3%
F1-Measure@2: 0.34%
-------
Precision@3: 0.72%
Recall@3: 0.69%
F1-Measure@3: 0.67%
-------
Precision@5: 1.98%
Recall@5: 2.09%
F1-Measure@5: 1.95%
-------
Precision@6: 3.59%
Recall@6: 4.53%
F1-Measure@6: 3.87%
-------
Precision@7: 6.71%
Recall@7: 13.53%
F1-Measure@7: 8.26%
-------
Mean Average Precision: 2.98%
